In [2]:
import requests


class ApeksAPI(object):
    URL = 'https://avtorvuz.orurinst.site'
    TOKEN = 'b41dfa22-0a72-477f-9f05-995b8409863a'
    APEKS_DEPT_ID = '4'  # ID of Department divisions type

def db_request(dbname):
    """DB request function without filter"""
    params = {"token": ApeksAPI.TOKEN, "table": dbname}
    response = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params
    )
    return response.json()["data"]


def db_filter_req(dbname, sqltable, sqlvalue):
    """Filtered DB request (DB table, filter, value)"""
    params = {
        "token": ApeksAPI.TOKEN,
        "table": dbname,
        "filter[" + sqltable + "]": str(sqlvalue),
    }
    response = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params
    )
    return response.json()["data"]


def plan_curriculum_disciplines(education_plan_id):
    """Getting disciplines info as dict (disc_ID:[disc_code:disc_name])"""
    disciplines = {}
    resp = db_filter_req(
        "plan_curriculum_disciplines", "education_plan_id", education_plan_id
    )
    for disc in resp:
        if disc["level"] == "3":
            disciplines[disc["id"]] = [
                disc["code"],
                db_filter_req("plan_disciplines", "id", disc["discipline_id"])[0]["name"],
            ]
    return disciplines


In [63]:
db_request('mm_work_programs')

[{'id': '4',
  'curriculum_discipline_id': '1483',
  'name': 'Основы теории государства и права',
  'description': 'Рабочая программа дисциплины "Основы теории государства и права" ',
  'authors': 'к.ф.н., доцент, Базавлук Леонид Михайлович\r\nк.п.н., доцент Баркалов Сергей Николаевич\r\nк.ю.н., доцент, Боева Милана Александровна\r\nк.ю.н.,  доцент Бондарев Александр Александрович\r\nк.ю.н., Ветрова Оксана Александровна\r\nДурнев Алексей Иванович\r\nк.ю.н., Клевцов Сергей Владимирович\r\nк.ю.н., Кондратьев Андрей Александрович\r\nк.ю.н., Кузминова Елена Юрьевна\r\nКузнецова Ирина Ивановна\r\nк.ю.н., Кураков Денис Владимирович\r\nМишина Юлия Вячеславовна\r\nНечушкина Кристина Александровна\r\nСемионкина Наталья Георгиевна\r\nк.ю.н., Смоляков Андрей Иванович\r\n',
  'reviewers_int': '',
  'reviewers_ext': 'Зиборов О.В., первый заместитель начальника Московского университета МВД России имени В.Я. Кикотя, доктор юридических наук, доцент, генерал-майор полиции;\r\nТодрик В.Н., заместитель н

In [66]:
db_request('mm_work_programs_fields')

[{'id': '1',
  'code': 'literature_primary',
  'name': 'Основная литература',
  'sort': '50',
  'group_id': '1',
  'type': '4'},
 {'id': '2',
  'code': 'literature_secondary',
  'name': 'Дополнительная литература',
  'sort': '60',
  'group_id': '1',
  'type': '4'},
 {'id': '3',
  'code': 'literature_methodical',
  'name': 'Научная продукция, подготовленная научно-педагогическим составом института',
  'sort': '70',
  'group_id': '1',
  'type': '4'},
 {'id': '8',
  'code': 'self_provision',
  'name': 'Обеспечение самостоятельной работы',
  'sort': '5',
  'group_id': '1',
  'type': '0'},
 {'id': '9',
  'code': 'task_works',
  'name': 'Задачи',
  'sort': '25',
  'group_id': '2',
  'type': '0'},
 {'id': '10',
  'code': 'practice',
  'name': 'Практикум',
  'sort': '70',
  'group_id': '2',
  'type': '0'},
 {'id': '11',
  'code': 'essay_works',
  'name': 'Тематика рефератов',
  'sort': '90',
  'group_id': '2',
  'type': '0'},
 {'id': '12',
  'code': 'course_works',
  'name': 'Тематика курсовых

In [6]:
data = db_filter_req('mm_work_programs', 'curriculum_discipline_id', 1483)

In [11]:
date_department = data[-1]["date_department"]
date = date_department.split("-")
date

['2019', '09', '11']

In [ ]:
if date_department:
    date = date_department.split("-")
    date_department = date[-1].date[-2].date[-3]
else:
    date_department = "Не заполнена"
date_department

In [8]:
document_department = data[-1]["document_department"]
document_department

'Протокол №17'

In [61]:
db_filter_req('mm_work_programs', 'id', 1483)

KeyError: 'data'

In [14]:
plan_disc_list = plan_curriculum_disciplines(133)

In [15]:
plan_disc_list

{'10463': ['Б.1.О.34',
  'Делопроизводство и режим секретности в органах внутренних дел'],
 '10464': ['Б.1.О.35', 'Основы управления в органах внутренних дел'],
 '10465': ['Б.1.В.ЭД.3.02',
  'Обеспечение прав человека в деятельности органов внутренних дел'],
 '10466': ['Б.1.О.25', 'Криминология'],
 '10550': ['Б.1.О.38', 'Административная деятельность полиции']}

In [65]:
db_filter_req('mm_sections', 'work_program_id', 3229)

[{'id': '1906',
  'work_program_id': '3229',
  'topic_id': None,
  'purposes': '1',
  'tasks': '',
  'place_in_structure': '',
  'knowledge': '',
  'skills': '',
  'abilities': '',
  'ownerships': ''}]

In [32]:
names = {}
for disc in plan_disc_list:
    try:
        names[' '.join(plan_disc_list[disc])] = db_filter_req('mm_work_programs', 'curriculum_discipline_id', disc)[0]['name']
    except IndexError:
        names[' '.join(plan_disc_list[disc])] = 'Программа отсутствует'

In [33]:
names

{'Б.1.О.34 Делопроизводство и режим секретности в органах внутренних дел': 'Делопроизводство и режим секретности в органах внутренних дел',
 'Б.1.О.35 Основы управления в органах внутренних дел': 'Основы управления в органах внутренних дел',
 'Б.1.В.ЭД.3.02 Обеспечение прав человека в деятельности органов внутренних дел': 'Обеспечение прав человека в деятельности органов внутренних дел',
 'Б.1.О.25 Криминология': 'Криминология',
 'Б.1.О.38 Административная деятельность полиции': 'Программа отсутствует'}

In [35]:
db_filter_req('plan_education_plans', 'id', 133)[0]['name']

'ПЛАН для тестирования'

In [69]:
wp_id = 2102
field_id = 30

params = {'token': ApeksAPI.TOKEN,
          'table': 'mm_work_programs_data',
          'filter[work_program_id]': wp_id,
          'filter[field_id]': field_id}
resp = requests.get(ApeksAPI.URL + '/api/call/system-database/get', params=params)

In [71]:
resp.json()

{'status': 1,
 'data': [{'id': '21700',
   'work_program_id': '2102',
   'field_id': '30',
   'data': 'Дисциплина относится к завершающему этапу подготовки специалиста и последующих дисциплин не имеет',
   'check': '1'}]}

In [5]:
payload = {"token": ApeksAPI.TOKEN, "table": "plan_education_specialties"}
request = requests.get(
    ApeksAPI.URL + "/api/call/system-database/get", params=payload
)

request.json()

{'status': 1,
 'data': [{'id': '71',
   'education_group_id': '29',
   'name': 'Правовое обеспечение национальной безопасности',
   'code': '01'},
  {'id': '72',
   'education_group_id': '29',
   'name': 'Правоохранительная деятельность',
   'code': '02'},
  {'id': '97',
   'education_group_id': '29',
   'name': 'Юриспруденция',
   'code': '01'},
  {'id': '98',
   'education_group_id': '29',
   'name': 'Обеспечение законности и правопорядка',
   'code': '02'}]}

In [8]:
payload = {
    "token": ApeksAPI.TOKEN,
    "table": "plan_education_plans",
    "filter[data_type]": "plan",
    "filter[education_specialty_id]": 98,
    "filter[active]": "1",
}
request = requests.get(
    ApeksAPI.URL + "/api/call/system-database/get", params=payload
)
request.json()

{'status': 1,
 'data': [{'id': '122',
   'parent_id': None,
   'data_type': 'plan',
   'name': '40.03.02 Обеспечение законности и правопорядка 2022 ГИБДД ФЗО',
   'name_short': '2022 ОЗиП ГИБДД ФЗО',
   'code': '',
   'generation': '3++',
   'education_level_id': '3',
   'education_specialty_id': '98',
   'education_specialization_id': '11',
   'qualification_id': '110',
   'credit_unit_value': '36',
   'credit_unit_count': '240',
   'status': '1',
   'approval_date': '2021-09-29',
   'approval_document': '1',
   'faculty_id': '24',
   'department_id': '15',
   'round': '0',
   'comments': '',
   'annotation': '',
   'check_settings': None,
   'custom_start_year': '2022',
   'user_id': '4',
   'active': '1',
   'settings': '{"educationTimeNorm":"","controlTypes":["1","9","8","6","3","2"],"controlTypesHours":{"1":"27","6":"10","2":"10"},"practiceHoursPercent":"0"}',
   'branch_id': None,
   'education_specialization_narrow_id': '2'},
  {'id': '144',
   'parent_id': None,
   'data_type':

In [16]:
db_filter_req('plan_semesters', 'education_plan_id', 122)[0]['start_date'].split('-')[0]

'2022'

In [20]:
def education_plans(education_specialty_id, year='0'):
    """Getting education plans with selected speciality"""
    payload = {
        "token": ApeksAPI.TOKEN,
        "table": "plan_education_plans",
        "filter[data_type]": "plan",
        "filter[education_specialty_id]": education_specialty_id,
        "filter[active]": "1",
    }
    request = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=payload
    )
    plans = {}
    if year == '0':
        for i in request.json()["data"]:
            plans[i.get("id")] = i.get("name")
        return plans
    else:
        for i in request.json()["data"]:
            if i.get("custom_start_year") == str(year):
                plans[i.get("id")] = i.get("name")
            elif i.get("custom_start_year") is None:
                if db_filter_req('plan_semesters', 'education_plan_id', i.get("id"))[0]['start_date'].split('-')[0] == str(year):
                    plans[i.get("id")] = i.get("name")
        return plans

In [27]:
plan_list = education_plans(71, 2021)

In [30]:
for plan in plan_list:
    print(plan_list[plan])

40.05.01 Правовое обеспечение национальной безопасности 2021
40.05.01 Правовое обеспечение национальной безопасности 2021 (иностранные слушатели)
40.05.01 Правовое обеспечение национальной безопасности 2021 ФЗО


In [31]:
def plan_department_disciplines(education_plan_id, department_id):
    """Getting department disciplines info as dict (disc_ID:[disc_code:disc_name])"""
    disciplines = {}
    resp = db_filter_req(
        "plan_curriculum_disciplines", "education_plan_id", education_plan_id
    )
    for disc in resp:
        if disc["level"] == "3" and disc["department"] == department_id:
            disciplines[disc["id"]] = [
                disc["code"],
                db_filter_req("plan_disciplines", "id", disc["discipline_id"])[0]["name"],
            ]
    return disciplines

In [35]:
plan_department_disciplines(133, 12)

KeyError: 'department'

In [36]:
db_filter_req(
        "plan_curriculum_disciplines", "education_plan_id", 133
    )

[{'id': '9863',
  'education_plan_id': '133',
  'code': 'Блок 1',
  'discipline_id': '2',
  'credit_unit_value': None,
  'hours': None,
  'semesters': None,
  'control_type_id': None,
  'level': '1',
  'type': None,
  'department_id': None,
  'left_node': '1',
  'right_node': '14',
  'settings': None},
 {'id': '9864',
  'education_plan_id': '133',
  'code': 'Б.1.О',
  'discipline_id': '328',
  'credit_unit_value': None,
  'hours': None,
  'semesters': None,
  'control_type_id': None,
  'level': '2',
  'type': None,
  'department_id': None,
  'left_node': '2',
  'right_node': '13',
  'settings': None},
 {'id': '10463',
  'education_plan_id': '133',
  'code': 'Б.1.О.34',
  'discipline_id': '34',
  'credit_unit_value': None,
  'hours': None,
  'semesters': None,
  'control_type_id': None,
  'level': '3',
  'type': None,
  'department_id': '14',
  'left_node': '3',
  'right_node': '4',
  'settings': None},
 {'id': '10464',
  'education_plan_id': '133',
  'code': 'Б.1.О.35',
  'discipline_i

In [34]:
params = {
    "token": ApeksAPI.TOKEN,
    "staff_id": 1,
    "month": '12',
    "year": '2021',
}
requests.get(
    ApeksAPI.URL + "/api/call/schedule-schedule/staff", params=params
).json()["data"]

{'departments': {'9': 'кафедра административного права и административной деятельности ОВД',
  '11': 'кафедра государственно-правовых дисциплин',
  '10': 'кафедра гражданско-правовых и экономических дисциплин',
  '22': 'кафедра иностранных и русского языков',
  '12': 'кафедра информационных технологий в деятельности ОВД',
  '13': 'кафедра криминалистики и предварительного расследования в ОВД',
  '17': 'кафедра огневой подготовки и деятельности ОВД в особых условиях',
  '14': 'кафедра оперативно-разыскной деятельности ОВД',
  '15': 'кафедра организации деятельности ГИБДД',
  '16': 'кафедра социально-философских дисциплин',
  '21': 'кафедра специальной подготовки',
  '18': 'кафедра уголовного права, криминологии и психологии',
  '19': 'кафедра уголовного процесса',
  '20': 'кафедра физической подготовки и спорта'},
 'staff': {'14': {'56': {'id': 56,
    'family_name': 'Савенков',
    'name': 'Юрий',
    'surname': 'Николаевич',
    'academicDegreeLevel': None,
    'academicDegreeType': N